In [1]:
from pipeNetwork import pipeNetwork
from containerClass import container
import numpy as np
from scipy.integrate import solve_ivp
import igraph
from calcTw import calcTw
from calcQ import calcQ
from containerClass import container
#from systemDefinitions import system,nodes,nodes1,pipeSections,pipeSections1
from systemDefinitions import system,pipeSections,pipeSections1, pipeSections3,orificeDiam3,pipeSections4,orificeDiam0,pipeSections0
from orificeForward import main_nozzle_forward, mass_critical
import matplotlib.pyplot as plt
from scipy.optimize import fsolve,root

IGRAPH DN-T 15 14 -- 
+ attr: agent (g), cyl_pressure (g), cyl_size (g), cyl_valve_type (g), discharge_time (g), D (v), M (v), MFR (v), P (v), P0 (v), T (v), T0 (v), calculated (v), name (v), rho (v), type (v), x (v), y (v), z (v), Cpl (e), D (e), Dtrp (e), Elb (e), H (e), L (e), MFR (e), P0i (e), Ptap (e), SV (e), Sch (e), Stee (e), Ttee (e), f (e)
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (4, 6), (5, 7), (5, 8), (8, 9), (7, 10), (10, 11), (7, 12), (12, 13), (6, 14)]
(0, 11, 8)
1 0 1 1 0
2 1 2 1 1
3 2 2 1 1
4 3 2 1 1
5 4 3 2 1
6 5 3 2 1
34 6 2 1 1
7 7 3 2 1
35 8 2 1 1
305 9 1 0 1
31 10 2 1 1
301 11 1 0 1
32 12 2 1 1
302 13 1 0 1
304 14 1 0 1


In [2]:
#!pip install --user --upgrade pixiedust

In [3]:
import pixiedust

Pixiedust database opened successfully


In [4]:
net = pipeNetwork()

In [5]:
net.addSystem(system)

In [6]:
net.addAllPipes(pipeSections0, orificeDiam0)

In [7]:
net.topoSummary()

IGRAPH DN-T 4 3 -- 
+ attr: agent (g), cyl_pressure (g), cyl_size (g), cyl_valve_type (g), discharge_time (g), D (v), M (v), MFR (v), P (v), P0 (v), T (v), T0 (v), calculated (v), name (v), rho (v), type (v), x (v), y (v), z (v), Cpl (e), D (e), Dtrp (e), Elb (e), H (e), L (e), MFR (e), P0i (e), Ptap (e), SV (e), Sch (e), Stee (e), Ttee (e), f (e)
[(0, 1), (1, 2), (2, 3)]
(0, 3, 3)
0 0 1 1 0
1 1 2 1 1
2 2 2 1 1
3 3 1 0 1


In [ ]:
_dt = 0.001
_initialBackP = 1e5
_initialMass = 32.2
_cylInitialTemp = 295
_agentInitialTemp = 294
_orificeDiam = 0.0066565
cont = [0] * len(tanks)
gamma = 1.4614 #from the theromo library for inergen
R = 8.3145 #* u.joule / u.kelvin / u.mole
MW = 0.0340669 #from the theromo library for inergen kg/mole

cont[firstTank] = container(agentInitialTemp = _agentInitialTemp,
             initialMass = _initialMass,
             cylInitialTemp = _cylInitialTemp,
             cylLength = 1.7,
             cylDiam = 0.267,
             nozzleDiam = _orificeDiam,
             wallThickness = 0.005,
             ambientTemp = 300,
             backPressure = _initialBackP,
             endTime = _dt,
             numTimeSteps = 2000)
cont[firstTank].solve()

In [ ]:
mdot0 = (cont[firstTank].ma()[0] - cont[firstTank].ma()[-1])/_dt

In [ ]:
mdot0

In [ ]:
cont[firstTank].Pt()[0]

In [ ]:
#total pressure inside the tank after dt time
P01 = cont[firstTank].Pt()[-1]
P01

In [ ]:
#total temperature inside the tank after dt time
T01 = cont[firstTank].Ta()[-1]

In [ ]:
#calculating the back pressure from equation 4 page 14 (isentropic process)
backPressure = P01 * (2/(gamma+1))**(gamma/(gamma-1))
print(backPressure)

In [ ]:
#calculating the mach number at the beginning of the first pipe after the tank valve 
def AAstar(M):
    val = A/Astar-1.0/M*((2.0/(gamma+1.0)*(1.0+(gamma-1.0)/2.0*M**2.0)))**((gamma+1.0)/(2.0*(gamma-1.0)))
    return val
    
pipeDiam = 0.0254
Astar = np.pi * _orificeDiam**2.0 /4.0
A = np.pi * pipeDiam**2.0 / 4.0

res = root(AAstar, 0.0001)
M = res['x'][0]
M

In [ ]:
def P0(M,P):
    return P*(1.0+(gamma-1.0)/2.0*M**2.0)**(gamma/(gamma-1))

P02 = P0(M,backPressure)
P02

In [ ]:
def T(T0,M):
    return T0 / (1+(gamma-1)/2.0*M**2)

T2 = T(T01,M)

In [ ]:
rho2 = backPressure / (R/MW) / T2

In [ ]:
#function to find the next tee, nozzle, tank, commonNode from a node and a pipe based on the direction of the pipe
#if the returned degree findNext()[1] is 1:tank or nozzle, 0:commonNode, 3:tee
def findNext(node, edge, graph, commonNode):
    _common = commonNode
    _edge = edge
    _node = node
    _nodeIDs = [node.index]
    _graph = graph
    if _node.index == _common:
        return _common, 0
    if _node.index == _edge.source:
        #move forward in the graph until the next tee
        _nextNodeID = _edge.target
        _nodeIDs.append(_nextNodeID)
        while _graph.vs[_nextNodeID].degree()==2 and _graph.vs[_nextNodeID].index != _common:
            _nextNodeID = _graph.vs[_nextNodeID].successors()[0].index
            _nodeIDs.append(_nextNodeID)
            
    elif _node.index == _edge.target:
        #move backwards in the graph until the next tee
        _nextNodeID = _edge.source
        _nodeIDs.append(_nextNodeID)
        while _graph.vs[_nextNodeID].degree()==2 and _graph.vs[_nextNodeID].index != _common:
            _nextNodeID = _graph.vs[_nextNodeID].predecessors()[0].index
            _nodeIDs.append(_nextNodeID)   
    _nodeType = _graph.vs[_nextNodeID].degree()
    if _nextNodeID == _common:
        _nodeType = 0
    return _nodeIDs, _nodeType

In [ ]:
g = net.t
n = findNext(g.vs[4],g.es[5],g, commonNode)
print(n[0])
print(n[1])

In [ ]:
#function to calculate the pressure drop between two nodes (from source to target)
#calculates all the properties on the path from the source to the target node 
#saves all the properties on the pipes and nodes. except for MFR and P0
#there should be only one path between source and target nodes

def calcNode(source, target, graph):
    #find all the nodes between the source and the target
    nodes = graph.get_shortest_paths(source.index, target.index,mode='ALL')
    for i in range(0,len(nodes)-1):
        edge = graph.es.select(_source = i, _target = i+1)
        #if we're going in the direction of the flow, then edge['L'] is positive, otherwise we need to multiply it by (-1)
        calcEdge = calcQ(edge['L'],...)

In [ ]:
#implementation of the forward pass
tank1 = net.t.vs[firstTank]
tank1Valve = net.t.vs[firstTank].successors()[0]
initialMdot0Guess = mdot0
g.es.select(_source = tank1.index, _target = tank1Valve.index)[0]['MFR'] = initialMdot0Guess #initial guess by considering a small dt for the container function
nextEdge = g.es.select(_source = tank1Valve.index, _target = tank1Valve.successors()[0].index)
if tank1Valve.index == commonNode:
    pass
    #calculate the properties for the common node and store it in that node
else:
    nextNode = findNext(tank1Valve,nextEdge[0],g,commonNode)
    #calculate the properties for the next node and store the properties on the next node
    calcNode(tank1Valve,nextNode[0][-1],g)
    while nextNode[0][-1] != commonNode:
        pass
        #find the oposite edge
        #find the next tank in the oposite edge direction
        #guess a mfr value for this tank
        #calculate the properties until the node
        #compare the P01 and P02 using the function ratio
        #if dp is good then assign average p0 to the node 
        #add the mfr from both inputs to the next edge
        #find the next node
        #calculate the properties for the next node and store the properties on that next node except MFR and P0, MFR and P0 should be saved only on the pipes

In [ ]:
#Backward pass implementation
g.vs[commonNode]['MFR'] = 1
totalMFR = g.vs[commonNode]['MFR']
nozzles = g.vs.select(_outdegree = 0)
for i in nozzles:
    i['MFR'] = totalMFR / len(nozzles)
    i['calculated'] = False
    #print(i['MFR'])

for noz in nozzles:
    if noz['calculated'] == False:
        print(findNext(noz,g.es.select(_target = noz.index)[0],g,commonNode))
        path = findNext(noz,g.es.select(_target = noz.index)[0],g,commonNode)
        calcNode(g.vs[path[0][0]],g.vs[path[0][-1]],g)

In [ ]:
#overall calculation until the final time
#read the pipe network information and populate the graph
finalTime = 120 #seconds, input from the user
t_old = 0
while t<finalTime
    
    #overall calculation for each dt (Backward + forward)
    while g.vs[commonNode]['MFR'] == 0:
        #use the ratio function to adjust the guess value for the firstTank
        #do the forward pass
        #do the backward pass
        #if dp from the backward and forward passes are < error then assign the MFR to the common node (at this point the calculation for dt is done)
        pass
    #update the tanks conditions based on the calculated values of MFR and backPressure
    t = t_old + dt

In [ ]:
C1 = 1e-5
C2 = 1e-5
Err = 0.1
#Assume 50% division at every node in the beginning
def divide(ratio2, P_01, P_02,c1,c2, error):
    del_P = P_02-P_01
    rat_old= ratio2
   
    if del_P > error:
        rat_new = rat_old* np.exp(-c1*del_P)
    elif del_P < -error:
        rat_new = 1 - np.exp(c2*del_P) + rat_old*np.exp(c2*del_P)
    
    return rat_new

#propagate mass flow rates downstream a tee
def propagateMFR(node, edge1, edge2, graph):
    P01 = edge1[P0i]
    P02 = edge2[P0i]
    MFR1 = edge1['MFR']
    MFR2 = edge2['MFR']
    MFR_total = MFR1 + MFR2
    ratio = MFR2 / MFR_total
    if np.abs(P01-P02)>Err:
        ratio_new = divide(ratio,P01,P02,C1,C2, Err)
        MFR1_new = (1 - ratio_new)*MFR_total 
        MFR2_new = ratio_new*MFR_total
        
        #distribute the MFR1_new to the downstream nozzles evenly
        nextNodeInEdge1Direction = findNext(node,edge1,graph,commonNode)
        if nextNodeInEdge1Direction[1] == 3:
            allPassesFromEdge1 = g.get_all_shortest_paths(nextNodeInEdge1Direction[0][-1],net.t.vs.select(_outdegree = 0))
            for noz in allPassesFromEdge1[][-1]:
                g.vs[noz]['MFR'] = MFR1_new / len(allPassesFromEdge1[][-1])
        else:
            graph.vs[nextNodeInEdge1Direction[0][-1]]['MFR'] = MFR1_new
           
        #distribute the MFR2_new to the downstream nozzles evenly
        nextNodeInEdge2Direction = findNext(node,edge2,graph,commonNode)
        if nextNodeInEdge2Direction[1] == 3:
            allPassesFromEdge2 = g.get_all_shortest_paths(nextNodeInEdge2Direction[0][-1],net.t.vs.select(_outdegree = 0))
            for noz in allPassesFromEdge2[][-1]:
                g.vs[noz]['MFR'] = MFR2_new / len(allPassesFromEdge2[][-1])
        else:
            graph.vs[nextNodeInEdge2Direction[0][-1]]['MFR'] = MFR2_new
            
        #set the 'calculated' property of all the nodes downstream the node to False    
        for i in graph.get_all_shortest_paths(node.index,graph.vs.select(_outdegree = 0)):
            for j in i:
                graph.vs[j]['calculated'] = False
                
        #set the P0i of all the edges downstream the node to 0
        for i in g.get_all_shortest_paths(node.index,g.vs.select(_outdegree = 0)):
            for k in range(0,len(i)-1):
                g.es.select(_source = i[k],_target = i[k+1])['P0i'] = 0
    else:
        node['calculated'] = True
        P0 = (P01+P02)/2.0
        node['P0']=P0
        edge3 = graph.es.select(_source = g.vs[4].predecessors()[0].index , _target = node.index)[0]
        previousNode = findNext(node,edge3,graph,commonNode)
        #calculate the properties for all the nodes until the next node backwards
        #set all the P0i for edges until the next node and set 'calculated' property of all nodes until the next node to True